Jupyter Notebook For ReadMyMind, A CS 125 @ Illinois MP7 Project by Isaac Park and Mihir Pandya.

In [35]:
import tweepy
import pandas as pd
from textblob import TextBlob
from datetime import datetime
import pprint
import numpy as np

username = 'realDonaldTrump' # working example: Donald Trump

auth = tweepy.OAuthHandler('1X5fCqPl7yVvYxQjQJwkvavFD', 'NXbTDPP3HxlXOL5dWdCegEP09odLAkxUWlyRvZqXxtAtdX597G')
auth.set_access_token('925495606931546112-mn3Hda41LsZhbYAKJtddL7TulRKucuj', 'lvCFqSLv5YvOGzCINH6JZ5cBI1CEkPKrRioBn5Iuec3Tt')
api = tweepy.API(auth)
    
tweets_df = pd.DataFrame({
    'Timestamp': (),
    'Likes': (),
    'Retweets': (),
    'Text': (),
    'Sentences': (),
    'Sentiment_Total': (),
    'Keywords': ()
})

tweets_df = tweets_df[['Timestamp', 'Likes', 'Retweets', 'Text', 'Sentences', 'Sentiment_Total', 'Keywords']]
    
recent_tweets = api.user_timeline(screen_name = username, count=100, tweet_mode="extended") # analyzing 100 tweets
for status in recent_tweets:
    test = status.full_text
    if test[:2] != 'RT': # removing retweets made by the user
        status_data = pd.Series([status.created_at, status.favorite_count, status.retweet_count, status.full_text], 
                                index=['Timestamp', 'Likes', 'Retweets', 'Text'])
    tweets_df = tweets_df.append(status_data, ignore_index = True)
    
tweets_df = tweets_df.drop_duplicates(subset='Text') # just in case, remove any duplicate tweets
tweets_df = tweets_df.astype('object')

keywords_dict = {}

for i in range(len(tweets_df)):
    content = tweets_df.iloc[i]['Text']
    if 'http' in content:
        j = content.index('http')
        content = content[:j] # cleaning text of the tweet by removing the link at the end and newline characters
    content = content.replace('\n', '')
    tweets_df.iloc[i]['Text'] = content
    
    blob = TextBlob(content)
    tweets_df.iloc[i]['Sentiment_Total'] = blob.sentiment.subjectivity
    sentiments = {}
    
    for sent in blob.sentences: # generating sentiment polarity values for each sentence in the tweet
        sentiments[str(sent)] = sent.sentiment.subjectivity
        
    tweets_df.iloc[i]['Sentences'] = sentiments # insert dictionary of sentence: sentiment value into dataframe
    
    tweets_df.iloc[i]['Timestamp'] = tweets_df.iloc[i]['Timestamp'].to_pydatetime() # convert pandas.tslib.Timestamp object to datetime
    
    # Keyword extraction goes here
    filtered_words = blob.noun_phrases
#     print(filtered_words)
    temp = []
    
    for element in filtered_words:
        for x in range(len(filtered_words)):
#             print(filtered_words[x])
#             print(element)
            if element != filtered_words[x] and element in filtered_words[x]:
                temp.append(element)
                #filtered_words = [x for x in filtered_words if x != element]
    parts_of_speech = blob.tags
    for element in temp:
        filtered_words = [x for x in filtered_words if x != element]
    
    for x in range(len(parts_of_speech)):
        if (parts_of_speech[x])[1] == 'NN':
            enter = True
            for element in filtered_words:
                if (parts_of_speech[x])[0] in element:
                    enter = False
            if enter:
                if x > 0 and (parts_of_speech[x - 1])[1] == 'PRP$':
                    filtered_words.append((parts_of_speech[x])[0])
    parenthesis = []
    paren_init = 0
    loc_begin = blob.find("(", paren_init)
    loc_end = blob.find(")", paren_init)
    
    while loc_end >= 0:
        parenthesis.append(blob[loc_begin:loc_end])
        paren_init = loc_end + 1
        loc_begin = blob.find("(", paren_init)
        loc_end = blob.find(")", paren_init)
    #print(parenthesis)
    
    for element in filtered_words:
        for pelement in parenthesis:
            if element in pelement.lower():
                filtered_words = [x for x in filtered_words if x != element]
#     tweets_df.iloc[i]['Keywords'] = filtered_words

    for word in filtered_words:
        separated = TextBlob(word).words
        for j in separated:
            j = j.strip()
            if j.isalpha() and len(j) > 2:
                if j in keywords_dict:
                    keywords_dict[j][0] += 1
                    keywords_dict[j][1] += tweets_df.iloc[i]['Likes']
                    keywords_dict[j][2] += tweets_df.iloc[i]['Retweets']
                    keywords_dict[j][3] += tweets_df.iloc[i]['Sentiment_Total']
#                     sentiment_sum = 0
#                     for sent in tweets_df.iloc[i]['Sentences']:
#                         if j in sent:
#                             sentiment_sum += tweets_df.iloc[i]['Sentences'][sent]
#                     keywords_dict[j][3] += sentiment_sum
                else:
                    keywords_dict[j] = [1, tweets_df.iloc[i]['Likes'], tweets_df.iloc[i]['Retweets'], tweets_df.iloc[i]['Sentiment_Total']]
#                     sentiment_sum = 0
#                     for sent in tweets_df.iloc[i]['Sentences']:
#                         if j in sent:
#                             sentiment_sum += tweets_df.iloc[i]['Sentences'][sent]
#                     keywords_dict[j][3] = sentiment_sum

for key in keywords_dict:
    keywords_dict[key][1] = int(keywords_dict[key][1] / keywords_dict[key][0])
    keywords_dict[key][2] = int(keywords_dict[key][2] / keywords_dict[key][0])
    keywords_dict[key][3] = keywords_dict[key][3] / keywords_dict[key][0]

keywords_df = pd.DataFrame.from_dict(keywords_dict, orient='index')
keywords_df.columns = ['Frequency', 'Avg. Likes', 'Avg. Retweets', 'Avg. Sentiment']
keywords_df.index.name = 'Keywords'
keywords_df.reset_index(inplace = True)
keywords_df = keywords_df.sort_values(['Frequency'], ascending = [False], na_position = 'last')
keywords_df = keywords_df[:30]

print(tweets_df['Timestamp'])

0     2017-12-10 21:18:40
1     2017-12-10 13:35:28
2     2017-12-10 13:30:44
3     2017-12-10 02:17:25
4     2017-12-09 23:47:55
5     2017-12-09 23:14:34
6     2017-12-09 22:01:44
7     2017-12-09 18:37:31
8     2017-12-09 18:19:31
9     2017-12-09 15:45:47
10    2017-12-09 13:21:24
11    2017-12-09 13:02:23
12    2017-12-09 12:52:18
13    2017-12-09 12:41:39
14    2017-12-09 06:06:03
15    2017-12-09 04:46:05
16    2017-12-09 01:04:16
17    2017-12-08 23:25:06
18    2017-12-08 19:02:27
19    2017-12-08 17:46:21
20    2017-12-08 15:18:27
21    2017-12-08 15:06:55
22    2017-12-08 13:11:43
23    2017-12-08 05:41:08
24    2017-12-08 05:36:09
25    2017-12-07 21:10:28
26    2017-12-07 20:52:49
27    2017-12-07 20:04:20
28    2017-12-07 16:16:19
29    2017-12-07 15:04:54
             ...         
66    2017-12-01 11:13:35
67    2017-12-01 11:03:35
68    2017-12-01 03:30:42
69    2017-12-01 00:42:21
70    2017-12-01 00:36:40
71    2017-11-30 21:50:33
72    2017-11-30 21:00:50
73    2017-1

Proposed method for keyword extraction:

1. Tokenize each word with part of speech tag. keep only proper nouns, nouns, adjectives, and verbs.
2. Score the nouns and proper nouns based on amount of surrounding adjectives and verbs (using more description tends to indicate importance).
3. Record frequency of each word; only keep words that occur above a certain number of times (frequency threshold). These will be our "keywords".
4. Put the list of keywords for each tweet into the 'Keywords' column of the dataframe.

Ideas for graphing the keywords/frequency/likes/retweets relationships:

1. y-axis: frequency, x-axis: keyword; simple bar graph of the top keywords

2. y-axis: likes/retweet count, x-axis: frequencies of keywords; scatter plot with each dot representing a keyword.

3. Simple pie chart to analyze the main content areas that said Twitter account comments on.




Written below is the function/method oriented version of the program. Different parts of the program are organized into multiple functions. -Isaac

In [76]:
import tweepy
import pandas as pd
from textblob import TextBlob
from datetime import datetime
import numpy as np
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

plotly.tools.set_credentials_file(username='Parkkeo1', api_key='qNMv1LXTLzwfGZ3EyN0U')

auth = tweepy.OAuthHandler('1X5fCqPl7yVvYxQjQJwkvavFD', 'NXbTDPP3HxlXOL5dWdCegEP09odLAkxUWlyRvZqXxtAtdX597G')
auth.set_access_token('925495606931546112-mn3Hda41LsZhbYAKJtddL7TulRKucuj', 'lvCFqSLv5YvOGzCINH6JZ5cBI1CEkPKrRioBn5Iuec3Tt')
api = tweepy.API(auth)

def tweet_collector(username): 
    tweets_df = pd.DataFrame({
        'Timestamp': (),
        'Likes': (),
        'Retweets': (),
        'Text': (),
        'Sentences': (),
        'Sentiment_Total': (),
        'Keywords': ()
    })

    tweets_df = tweets_df[['Timestamp', 'Likes', 'Retweets', 'Text', 'Sentences', 'Sentiment_Total', 'Keywords']]

    recent_tweets = api.user_timeline(screen_name = username, count=500, tweet_mode="extended") # analyzing 100 tweets
    for status in recent_tweets:
        test = status.full_text
        if test[:2] != 'RT': # removing retweets made by the user
            status_data = pd.Series([status.created_at, status.favorite_count, status.retweet_count, status.full_text], 
                                    index=['Timestamp', 'Likes', 'Retweets', 'Text'])
        tweets_df = tweets_df.append(status_data, ignore_index = True)

    tweets_df = tweets_df.drop_duplicates(subset='Text') # just in case, remove any duplicate tweets
    tweets_df = tweets_df.astype('object')
    
    return tweets_df

def keyword_data(tweets_df):
    keywords_dict = {}

    for i in range(len(tweets_df)):
        content = tweets_df.iloc[i]['Text']
        if 'http' in content:
            j = content.index('http')
            content = content[:j] # cleaning text of the tweet by removing the link at the end and newline characters
        content = content.replace('\n', '')
        tweets_df.iloc[i]['Text'] = content

        blob = TextBlob(content)
        tweets_df.iloc[i]['Sentiment_Total'] = blob.sentiment.subjectivity
        sentiments = {}

        for sent in blob.sentences: # generating sentiment polarity values for each sentence in the tweet
            sentiments[str(sent)] = sent.sentiment.subjectivity

        tweets_df.iloc[i]['Sentences'] = sentiments # insert dictionary of sentence: sentiment value into dataframe

        tweets_df.iloc[i]['Timestamp'] = tweets_df.iloc[i]['Timestamp'].to_pydatetime() # convert pandas.tslib.Timestamp object to datetime

        # Keyword extraction goes here
        filtered_words = blob.noun_phrases
    #     print(filtered_words)
        temp = []

        for element in filtered_words:
            for x in range(len(filtered_words)):
    #             print(filtered_words[x])
    #             print(element)
                if element != filtered_words[x] and element in filtered_words[x]:
                    temp.append(element)
                    #filtered_words = [x for x in filtered_words if x != element]
        parts_of_speech = blob.tags
        for element in temp:
            filtered_words = [x for x in filtered_words if x != element]

        for x in range(len(parts_of_speech)):
            if (parts_of_speech[x])[1] == 'NN':
                enter = True
                for element in filtered_words:
                    if (parts_of_speech[x])[0] in element:
                        enter = False
                if enter:
                    if x > 0 and (parts_of_speech[x - 1])[1] == 'PRP$':
                        filtered_words.append((parts_of_speech[x])[0])
        parenthesis = []
        paren_init = 0
        loc_begin = blob.find("(", paren_init)
        loc_end = blob.find(")", paren_init)

        while loc_end >= 0:
            parenthesis.append(blob[loc_begin:loc_end])
            paren_init = loc_end + 1
            loc_begin = blob.find("(", paren_init)
            loc_end = blob.find(")", paren_init)
        #print(parenthesis)

        for element in filtered_words:
            for pelement in parenthesis:
                if element in pelement.lower():
                    filtered_words = [x for x in filtered_words if x != element]
    #     tweets_df.iloc[i]['Keywords'] = filtered_words

        for word in filtered_words:
            separated = TextBlob(word).words
            for j in separated:
                j = j.strip()
                if j.isalpha() and len(j) > 2:
                    if j in keywords_dict:
                        keywords_dict[j][0] += 1
                        keywords_dict[j][1] += tweets_df.iloc[i]['Likes']
                        keywords_dict[j][2] += tweets_df.iloc[i]['Retweets']
                        keywords_dict[j][3] += tweets_df.iloc[i]['Sentiment_Total']
                    else:
                        keywords_dict[j] = [1, tweets_df.iloc[i]['Likes'], tweets_df.iloc[i]['Retweets'], tweets_df.iloc[i]['Sentiment_Total']]

    for key in keywords_dict:
        keywords_dict[key][1] = int(keywords_dict[key][1] / keywords_dict[key][0])
        keywords_dict[key][2] = int(keywords_dict[key][2] / keywords_dict[key][0])
        keywords_dict[key][3] = keywords_dict[key][3] / keywords_dict[key][0]
        
    return keywords_dict

def to_dataframe(keywords_dict):
    keywords_df = pd.DataFrame.from_dict(keywords_dict, orient='index')
    keywords_df.columns = ['Frequency', 'Avg. Likes', 'Avg. Retweets', 'Avg. Sentiment']
    keywords_df.index.name = 'Keywords'
    keywords_df.reset_index(inplace = True)
    keywords_df = keywords_df.sort_values(['Frequency'], ascending = [False], na_position = 'last')
    keywords_df = keywords_df[:30]
    
    return keywords_df

def scatter_plot(keywords_df):
    
    trace = go.Scatter(
        x = list(keywords_df['Avg. Sentiment']),
        y = list(keywords_df['Avg. Likes']),
        mode = 'markers'
    )
    
    data = [trace]
    
    return py.iplot(data, filename='basic-scatter')

def bar_chart(keywords_df):
    data = [go.Bar(
            x = list(keywords_df['Keywords']),
            y = list(keywords_df['Avg. Sentiment'])
    )]

    return py.iplot(data, filename='basic-bar')

# def time_series(tweets_df, key_df):
#     freq_list = []
    
#     for i in range(len(tweets_df)):
#         status = tweets_df.iloc[i]['Text']
#         freq = status.lower().split().count(key_df['Keywords'].iloc[0])
#         freq_list.append(freq)
    
    
#     data = [go.Scatter(
#           x = tweets_df['Timestamp'],
#           y = freq_list)]

#     return py.iplot(data)

def pie_graph(keywords_df):
    labels = keywords_df['Keywords'][:10]
    values = keywords_df['Frequency'][:10]

    trace = go.Pie(labels=labels, values=values)

    return py.iplot([trace], filename='basic_pie_chart')

def ReadMyMind_main(username):
    data_df = tweet_collector(username)
    keywords = keyword_data(data_df)
    key_df = to_dataframe(keywords)
    return pie_graph(key_df)



In [77]:
ReadMyMind_main('BernieSanders')

In [78]:
ReadMyMind_main('HillaryClinton')

In [79]:
ReadMyMind_main('realDonaldTrump')